<a href="https://colab.research.google.com/github/yiiyama/qc-workbook-lecturenotes/blob/branch-2024/2024_06_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VQEで水素分子の基底エネルギーを求める

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$

VQE (variational quantum eigensolver; 変分量子固有値ソルバー)を使って実際の量子化学の問題を解いてみましょう。

VQEは2014年に発表された手法（[Peruzzo et al.](https://doi.org/10.1038/ncomms5213)）で、超伝導型量子コンピュータを使って水素分子などの小さい分子の基底状態エネルギーを求める問題に応用されたのは2017年の[Kandala et al.](https://doi.org/10.1038/nature23879)が始めてです。2017年というと量子コンピュータ業界においては太古の出来事ですが、それでも実際の学術論文の題材になった問題を実機で試すのは面白いのではないかと思います。

このコースは量子化学の講義ではないので、今回は問題のセットアップには深入りしません。幸い、Qiskit Nature（Qiskitの自然科学計算関連のモジュール）にいろいろなツールが組み込まれており、扱いたい分子の情報だけから量子コンピュータで期待値を測定すべき演算子を導出するところまでは自動でできます。この実習では、

- VQEを行うための変分量子回路を書く
- 渡された演算子の期待値を計算し、量子回路が作る量子状態のエネルギー期待値を計算する関数を書く
- Qiskit Runtimeを使って、実機での計算結果をもとにエネルギーを最小化するように変分回路のパラメータを調整していく

という部分を扱います。

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

!pip install --prefer-binary pyscf

runtime_config_path = '/content/gdrive/MyDrive/qiskit-ibm.json'

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator, SamplerV2 as AerSampler
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2 as RuntimeEstimator
from qiskit_nature import settings
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper

settings.use_pauli_sum_op = False

from qc_workbook.utils import operational_backend
from qc_workbook.show_state import show_state

## （背景知識）水素分子を解くとは？

量子化学の背景とVQEとのつながりについての詳しい説明は、この実習の発想元になっている[2022年のIBM Quantum Challengeのexercise 4](https://github.com/qiskit-community/ibm-quantum-spring-challenge-2022/blob/main/exercise4/04.quantum_chemistry.ipynb)（[日本語版](https://github.com/qiskit-community/ibm-quantum-spring-challenge-2022/blob/main/exercise4/04.quantum_chemistry-ja.ipynb)）が参考になります。ここでは最低限の説明をします。

今やりたいことは、ある分子の基本情報（構成する原子核種と数、電子の数、原子核の位置）を与えられた時に、その分子のエネルギー基底状態のエネルギー固有値を求めることです。ここで、原子核は電子に比べて遥かに重いことを利用し、原子核の位置を手で与えるパラメータとしています（Born-Oppenheimer approximation）。この基底状態エネルギーが求められれば、例えば今度は原子核の位置を動かしながら同じ計算を繰り返すことによって、最も基底状態エネルギーの低くなる原子核の配置（＝自然界で実現される分子のエネルギー基底状態の良い近似）を探すことなどができます。

### 分子軌道と$n$電子系の波動関数

原子単体に対しては、電子の軌道（エネルギー固有波動関数）$\{\chi_j(\mathbf{r})\}_j$とそれらに対応するエネルギーを、古典コンピュータでも精度良く計算することができます。ここで$\mathbf{r}$は電子の位置を表し、$j$はエネルギー固有状態のインデックスです。さらに分子中の一電子軌道（Molecular orbital）$\phi_m(\mathbf{r})$（$m$は軌道のインデックス）も、分子を構成する原子における軌道の重ね合わせとして

$$
\phi_m(\mathbf{r}) = \sum_j D_{mj}\chi_j(\mathbf{r})
$$

のように表現できます。

理論的には原子軌道も分子軌道も無数に存在しますが、今の問題のような場合、固有エネルギーの高い軌道はほとんど影響を持たないので、有限個の$\phi_m$のみを扱います。また、電子にはスピンの自由度があり、内部に量子ビットのような二次元系を持ちますが、それも軌道の一部とみなしていいので（spin orbitals）、結局、固有エネルギーで順序づけして下から$2M$個の分子軌道を考えることになります。

一般に分子には複数の電子がありますが、パウリの排他律から、各軌道には最大1個の電子しか入りません。従って、$n$電子系の波動関数は$n$個の占有されている軌道の掛け算で表現されます。ただし、同じくパウリの排他律の帰結として、全系の波動関数は電子の交換について反対称になるので、単純な掛け算ではなくSlater行列式$| \phi_{m_1} \phi_{m_2} \dots \phi_{m_n} | (\mathbf{r}_1, \mathbf{r}_2, \dots, \mathbf{r}_n) $が用いられます。一般の$n$電子系の状態はこのSlater行列式を基底とする線形空間の元と対応づけられて、

$$
\psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2, \dots,\mathbf{r}_n) = \sum_{m_1, m_2,\dots,m_n} C_{m_1, m_2, \dots, m_n} | \phi_{m_1}\phi_{m_2} \dots \phi_{m_n} | (\mathbf{r}_1, \mathbf{r}_2, \dots, \mathbf{r}_n)
$$

と書けます。特に、$n$電子系のエネルギー固有状態も上のようなSlater行列式の重ね合わせになります（電子同士の相互作用によって、全体のエネルギー固有状態は一電子のエネルギー固有状態の積にはなりません）。したがって、エネルギー固有状態や固有値を求めることは上の係数$C_{m_1, m_2, \dots, m_n}$を求めることに帰着します。ところが、軌道$\phi_m$が$2M$個あり、電子が$n$個あれば、系は$\binom{2M}{n}$次元となり、古典コンピュータでの計算はすぐに破綻してしまいます。そこで量子コンピュータの出番となるわけです。

### 第二量子化表現とJordan-Wigner変換

分子軌道の問題を量子コンピュータで扱う場合、系の状態を$n$体の波動関数$\psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2, \dots,\mathbf{r}_n)$で表現するよりも、$2M$個の軌道の占有・非占有で表現（第二量子化）する方が自然なマッピングとなります。その場合、ハミルトニアンは分子軌道ごとに定義される「生成・消滅演算子」$\hat{a}_m, \hat{a}_m^{\dagger}$を使って書かれ、

$$
\hat{H} =\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s}
+\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N}
$$

となります。ここで$h_{rs}$、$g_{pqrs}$、$E_N$は全て分子の構造から決まる（古典的に計算できる）定数で、それぞれ一電子のエネルギー、二電子間の電気的な反発のエネルギー、原子核同士の反発エネルギーに対応します。

生成・消滅演算子は、軌道$m$が占有されていない状態を$\ket{u}_m$、されている状態を$\ket{o}_m$とすると、

$$
\begin{align}
\hat{a}_m \ket{u}_m & = 0, \\
\hat{a}_m \ket{o}_m & = \ket{u}_m, \\
\hat{a}_m^{\dagger} \ket{u}_m & = \ket{o}_m, \\
\hat{a}_m^{\dagger} \ket{o}_m & = 0
\end{align}
$$

のように作用します。

このような占有・非占有の2次元系は、課題3のSchwingerモデルの問題にもあった通り、そのまま量子ビットに対応づけられます。つまり、$2M$個の軌道の状態を$2M$ビットの量子レジスタで表現できます。Schwingerモデルの問題ではサイトに粒子が存在する状態を$\ket{0}$、存在しない状態を$\ket{1}$に対応づけしましたが、今回はQiskitの習わしに従って、軌道が占有されている状態を$\ket{1}$、占有されていない状態を$\ket{0}$で表します。すると、$\hat{a}$と$\hat{a}^{\dagger}$はそれぞれ$\frac{1}{2}(X + iY)$と$\frac{1}{2}(X - iY)$に対応します。

$$
\begin{align}
\frac{1}{2}(X + iY) \ket{0} & = \frac{1}{2} (\ket{1} - \ket{1}) = 0, \\
\frac{1}{2}(X + iY) \ket{1} & = \frac{1}{2} (\ket{0} + \ket{0}) = \ket{0}, \\
\frac{1}{2}(X - iY) \ket{0} & = \frac{1}{2} (\ket{1} + \ket{1}) = \ket{1}, \\
\frac{1}{2}(X - iY) \ket{1} & = \frac{1}{2} (\ket{0} - \ket{0}) = 0.
\end{align}
$$

ただし、異なる軌道$j$と$k$の生成・消滅演算子が「反交換」しなければいけない（$a_r a_s = -a_s a_r$）という物理的要請があるので、実際には軌道$j$の生成・消滅演算子は

$$
\begin{align}
a_j \rightarrow I_{2M-1} \otimes I_{2M-2} \otimes \dots \otimes \frac{X_j + iY_j}{2} \otimes Z_{j-1} \otimes \dots \otimes Z_{0}, \\
a_j^{\dagger} \rightarrow I_{2M-1} \otimes I_{2M-2} \otimes \dots \otimes \frac{X_j - iY_j}{2} \otimes Z_{j-1} \otimes \dots \otimes Z_{0}
\end{align}
$$

のように、$j$より小さい軌道のスロットに$Z$演算子を挿入したような量子レジスタの演算に対応します。このような対応づけをJordan-Wigner変換といいます。実はSchwingerモデルの問題でも同じ操作をしていますが、説明を割愛していました。

Qiskit Natureのさまざまなクラスを利用すると、分子の核種と三次元位置を入力するだけで、ハミルトニアンの定数部分を計算し、生成・消滅演算子をJordan-Wigner変換して、ハミルトニアンをパウリ積（$X, Y, Z$ゲートと恒等演算$I$の組み合わせ）の和にしてくれます。

In [ ]:
molecule = MoleculeInfo(
    ['H', 'H'],
    # 原子核の座標（単位Å）
    [(0.0, 0.0, -0.3695), (0.0, 0.0, 0.3695)],
    charge=0,
    multiplicity=1  # = 2*spin + 1
)

# sto3g: 原子軌道の計算手法
driver = PySCFDriver.from_molecule(molecule, basis='sto3g')

# 解くべき問題を生成
problem = driver.run()

# 内殻軌道を固定し、問題の次元を削減
problem = FreezeCoreTransformer().transform(problem)

# 問題を表すハミルトニアンを第二量子化形式で表示
hamiltonian = problem.hamiltonian.second_q_op()
print(hamiltonian)

上で`basis="sto3g"`で[STO-3G](https://en.wikipedia.org/wiki/STO-nG_basis_sets)という原子軌道の計算（展開）方法を指定しています。STO-3Gでは最低限必要な軌道しか計算に含まないので、$2M=2n$となり、水素分子の場合は$n=2$なので4分子軌道のハミルトニアンが導出されます。軌道の番号と実際の分子軌道との対応づけは

- $0$から$n-1$番軌道がスピン$\alpha$の軌道
- $n$から$2n-1$番軌道がスピン$\beta$の軌道

で、それぞれエネルギーの低い順に並んでいます。$\alpha$と$\beta$は、何か適当な方向を基底に決めた時のスピンの二つの固有状態を表します。

上のハミルトニアンのプリントアウトでは、例えば`+_0`は第0軌道の生成演算子、`-_3`は第3軌道の消滅演算子を表します。

次にこのハミルトニアンをJordan-Wigner変換し、これまで講義で扱ってきたSparsePauliOp（パウリ積の和）オブジェクトに落とし込みます。

In [ ]:
ham_op = JordanWignerMapper().map(hamiltonian)
print(ham_op)

## 量子計算部分

### 期待値の計算

量子コンピュータにおける観測量の期待値の計算方法についてこれまで何度か触れてきましたが、今回はより複雑な計算となるので、ステップを追って確認してみましょう。

#### おさらい：$X, Y, Z$の期待値

適当な1ビット量子回路で作られる量子状態での$X, Y, Z$の期待値を計算しましょう。

量子コンピュータで観測量の期待値を計算するには、まず観測量を演算子で表し、その固有ベクトルと固有値を求め、量子回路の終状態に対して固有ベクトルが計算基底に移るような基底変換をし、測定の結果得られる計算基底の確率分布と固有値を掛け合わせるのでした。

定義上、$Z$の固有ベクトルとは計算基底そのもので、固有値は$\pm 1$（$Z\ket{0} = \ket{0}, Z\ket{1} = -\ket{1}$）です。したがって、量子回路の終状態をそのまま測定し、0が出る確率$P_0$から1が出る確率$P_1$を引けば、$Z$の期待値が求まります。式で書けば

$$
\langle Z \rangle = P_0 - P_1
$$

です。

$X$の固有値$\pm 1$の固有ベクトルは$\frac{1}{\sqrt{2}} (\ket{0} \pm \ket{1})$なので、アダマールゲートで基底変換ができます。

$Y$の固有ベクトルは$\frac{1}{\sqrt{2}}(\ket{0} \pm i\ket{1})$なので、$S^{\dagger} = P(-\pi/2)$とアダマールゲートで基底変換ができます。$S^{\dagger}$ゲートのqiskitでの記法は`circuit.sdg(0)`です。

In [ ]:
simulator = AerSimulator()

In [ ]:
# ランダムな1ビット回路
circuit = random_circuit(num_qubits=1, depth=3)

# 観測量 X, Y, Z
observables = [SparsePauliOp(op) for op in ['X', 'Y', 'Z']]

# 状態ベクトルから計算する厳密解
exact = []
for obs in observables:
    exact.append(
        Statevector(circuit).expectation_value(obs).real
    )

In [ ]:
circuits = []

# Xの期待値を計算する回路
circuit_x = circuit.copy()

### EDIT BELOW ###

### EDIT ABOVE ###

circuits.append(circuit_x)

# Yの期待値を計算する回路
circuit_y = circuit.copy()

### EDIT BELOW ###

### EDIT ABOVE ###

circuits.append(circuit_y)

# Zの期待値を計算する回路
circuit_z = circuit.copy()

### EDIT BELOW ###

### EDIT ABOVE ###

circuits.append(circuit_z)

# トランスパイルしてSamplerで実行
isa_circuits = transpile(circuits, backend=simulator, optimization_level=1)

shots = 10000

sampler = AerSampler()
sampler_result = sampler.run(isa_circuits, shots=shots).result()

# 結果のcountsから期待値を計算
observed_manual = []
for res in sampler_result:
    counts = res.data.meas.get_counts()
    # countsは {'0': number_of_0s, '1': number_of_1s}
    ### EDIT BELOW ###

    ### EDIT ABOVE ###

observed_manual = np.array(observed_manual)
probs = (observed_manual + 1.) * 0.5
observed_err_manual = np.sqrt(probs * (1. - probs) / shots) * 2.

plt.bar(np.arange(3), exact, label='Exact')
plt.errorbar(np.arange(3), observed_manual, yerr=observed_err_manual, xerr=0, ecolor='k', marker='+', mfc='k',
             mec='k', linestyle='none', label='Observed')
plt.xticks(np.arange(3), ['X', 'Y', 'Z'], fontsize=16)
plt.axhline(0., color='k', linestyle='--')
plt.legend(fontsize=16);

上のセルを何度か実行してみて、毎回`observed`と`exact`が一致することを確認してください。

Estimatorというクラスを使うと期待値が自動で計算できます。

In [ ]:
# circuitをトランスパイルし、論理量子ビットの物理量子ビットへの割当てをSparsePauliOpに適用する
isa_circuit = transpile(circuit, backend=simulator, optimization_level=1)
isa_observables = [obs.apply_layout(isa_circuit.layout) for obs in observables]

# Estimatorで期待値を計算する
estimator = AerEstimator(options={'default_precision': 0.01})

# Estimatorの使い方：run() に (回路、観測量のリスト) のリストを渡す
result = estimator.run([
    (isa_circuit, isa_observables)
]).result()

# resultはlist-likeなオブジェクトで、run()に渡した入力リストに対応する
observed = result[0].data.evs
observed_err = result[0].data.stds

plt.bar(np.arange(3), exact, label='Exact')
plt.errorbar(np.arange(3), observed, yerr=observed_err, xerr=0, ecolor='k', marker='+', mfc='k',
             mec='k', linestyle='none', label='Observed')
plt.xticks(np.arange(3), ['X', 'Y', 'Z'], fontsize=16)
plt.axhline(0., color='k', linestyle='--')
plt.legend(fontsize=16);

#### $I$の期待値

恒等演算子を観測量としたとき、その期待値はいくらになるでしょうか？

#### $ZIZI$の期待値

今度は4ビットの回路で観測量$ZIZI$の期待値を計算します。Estimatorを使うとトリビアルですが、手計算では何をしたらいいでしょうか？

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
observable = SparsePauliOp('ZIZI')
exact = Statevector(circuit).expectation_value(observable).real

observed = None

# AerEstimatorを使って期待値observedを計算してください。
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

print(f'Estimator: {observed}')
print(f'Exact: {exact}')

In [ ]:
# 手計算の場合

obs_circuit = circuit.copy()
obs_circuit.measure_all()
isa_obs_circuit = transpile(obs_circuit, backend=simulator)

sampler = AerSampler()
shots = 100000
result = sampler.run([isa_obs_circuit], shots=shots).result()

counts = result[0].data.meas.get_counts()

observed_manual = 0.

for key, value in counts.items():
    ##################
    ### EDIT BELOW ###
    ##################

    ##################
    ### EDIT ABOVE ###
    ##################
    pass

print(f'Manual: {observed_manual}')

#### $ZIZI + 2IZII + 3XXYY$の期待値

今度の観測量はパウリ積の和です。量子回路ではこれを一度に計算することができませんが、和の期待値は期待値の和なので、それぞれの項について独立に期待値を計算し、最後に係数をかけて足し合わせれば同じ答えが得られます。量子回路はいくつ必要でしょうか？

（Estimatorを使うと上のような考察は全部Qiskitが勝手にやってくれてしまいますが）

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
observable = SparsePauliOp(['ZIZI', 'IZII', 'XXYY'], [1., 2., 3.])
exact = Statevector(circuit).expectation_value(observable).real

isa_circuit = transpile(circuit, backend=simulator)
estimator = AerEstimator()
result = estimator.run([(isa_circuit, [observable])]).result()
observed = result[0].data.evs[0]

print(f'QC: {observed}')
print(f'Exact: {exact}')

#### Estimatorの利用と水素分子ハミルトニアンの期待値

上の`ham_op`の期待値を、適当な4ビット回路について計算しましょう。

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
exact = Statevector(circuit).expectation_value(ham_op).real

# 回路からEstimatorを使って計算する期待値
isa_circuit = transpile(circuit, backend=simulator)
estimator = AerEstimator()
result = estimator.run([(isa_circuit, [ham_op])]).result()
observed = result[0].data.evs[0]

print(f'QC: {observed}')
print(f'Exact: {exact}')

### 変分量子回路の設計と実装

回路とハミルトニアンが与えられた時にエネルギーを計算する手法が確立できたので、次は最低エネルギー状態を作る回路を構成することを考えます。もちろん、そんな回路が最初からわかっているわけではないので、代わりに適当なパラメータつき量子回路を設計し、その回路におけるエネルギー期待値が最も小さくなるようにパラメータの値を更新していく、つまり変分量子回路の手法を使います。

まず、近似的な低エネルギー状態を作ります。「あたり」をつけてからパラメータ付き回路部分で微調整をするようにすることで、解の探索を速くできます。

最低エネルギー状態の第0近似はいわゆるHartree-Fock状態、つまり電子が軌道を下から占有した状態です。今の場合、一番エネルギーの低い軌道は$\phi_0$と$\phi_2$（$\alpha$と$\beta$スピンの最低エネルギー軌道）なので、

$$
\psi_{\mathrm{HF}} = |\phi_0 \phi_2|.
$$

Jordan-Wigner変換で対応づけた量子レジスタで表すと

$$
\ket{\psi_{\mathrm{HF}}} = \ket{1010}
$$

となります。

In [ ]:
hf_circuit = QuantumCircuit(4)

# hf_circuitを|0000>からHartree-Fock状態|1010>を作る回路にしてください。量子ビットは右から数えることに注意。
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

show_state(hf_circuit, state_label=None, binary=True);

パラメータ付き回路部分の設計には最先端の理論に基づく後ろ盾と勘が必要ですが、今は手始めに業界でTwoLocalなどと呼ばれるパターンを使いましょう。

In [ ]:
twolocal = TwoLocal(num_qubits=4, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx',
                    entanglement='full', reps=2, skip_final_rotation_layer=True, insert_barriers=True)
twolocal.decompose().draw('mpl')

ここではTwoLocalのパターンを2「レイヤー」繰り返しています。全ての$R_y$と$R_z$ゲートに独立なパラメータをあてがうので、全部で16個の自由パラメータを持つ回路になります。

基底状態の近似とパラメータ付き回路を組み合わせてアンザッツとします。

In [ ]:
ansatz = hf_circuit.compose(twolocal, inplace=False)
ansatz.draw('mpl')

### エネルギー最小化

VQEでエネルギーを最小化するパラメータ値を探索します。ハミルトニアンの期待値をロスの値として返すコスト関数を定義し、scipyのminimizeという関数を使ってロスを最小化します。最適化アルゴリズムにはCOBYLAを使います。最小化の途中経過を`history`というdictに記録します。


In [ ]:
def cost_function(params, ansatz, hamiltonian, estimator, history):
    # Estimator.run()にパラメータ付き回路を渡す場合、各要素は(回路、観測量、パラメータ値)となる
    result = estimator.run([
        (ansatz, [hamiltonian], [params])
    ]).result()
    energy = result[0].data.evs[0]

    history["iters"] += 1
    history["prev_vector"] = params
    history["cost_history"].append(energy)
    if history['iters'] % 100 == 0:
        print(f"Iters. done: {history['iters']} [Current cost: {energy}]")

    return energy

# 各パラメータを[0, π)の乱数に初期化
init = np.random.default_rng().random(ansatz.num_parameters) * np.pi

estimator = AerEstimator()
isa_ansatz = transpile(ansatz, backend=simulator)
isa_hamiltonian = ham_op.apply_layout(isa_ansatz.layout)
history = {'iters': 0, 'prev_vector': None, 'cost_history': []}
res = minimize(cost_function, init, args=(isa_ansatz, isa_hamiltonian, estimator, history),
               method='cobyla', options={'tol': 1.e-4, 'maxiter': 2000})

plt.plot(history['cost_history'])

ちなみに、今回も小さな系を扱っているので基底状態エネルギーの厳密解も計算できます。

In [ ]:
# SparsePauliOpをnumpyのアレイに変換
ham_matrix = ham_op.to_operator().data
# Numpyのlinalgモジュール中のエルミート行列の固有ベクトルを計算する関数eigh
eigvals, eigvecs = np.linalg.eigh(ham_matrix)
# eigvalsは最小から最大までソートされた固有値のアレイ
exact_ground_state_energy = eigvals[0]

plt.plot(history['cost_history'])
plt.axhline(exact_ground_state_energy, color='k', linestyle='--')

## Qiskit Runtime

これまでローカルのシミュレータ（AerSimulator）のみを使って計算をしてきましたが、次にクラウド上のリソースを使うことを考えてみましょう。




In [ ]:
service = QiskitRuntimeService(filename=runtime_config_path)

空いている実機を探して使います。

In [ ]:
# 今一番空いているバックエンド
backend = service.least_busy(simulator=False, operational=True)
print(backend.name)

VQEのような量子・古典ハイブリッド計算では、量子と古典の計算リソースを交互に使うことが多くあります。この量子計算部分で、これまで講義で主に扱ってきたように回路をクラウドに投げてキューに入り、結果が帰ってくるまで待つということを繰り返していると、計算にあまりにも時間がかかってしまいます。そこで、IBM QuantumではQiskit Runtime Sessionという仕組みが用意されています。

セッションを利用すると、ユーザーが投げる最初のジョブはキューに入りますが、それが実行された時点からセッションが閉じられるまでの間はジョブが遅延無しで実行されるようになります。ただし、ジョブを実行してからインアクティブな時間が5分以上経過するとセッションが一旦停止され、次のジョブは再びキューで待つことになります。また、セッションはユーザーが指定するか開始から8時間経過するかのいずれかで閉じられます。

セッションの開始と終了はPythonの`with`構文を使って記述できます。

In [ ]:
with Session(service=service, backend=backend) as session:
    estimator = RuntimeEstimator(session=session)

    history = {'iters': 0, 'prev_vector': None, 'cost_history': []}

    estimator = RuntimeEstimator(session=session)
    estimator.options.default_shots = 10000
    isa_ansatz = transpile(ansatz, backend=backend)
    isa_hamiltonian = ham_op.apply_layout(isa_ansatz.layout)
    res = minimize(cost_function, init, args=(isa_ansatz, isa_hamiltonian, estimator, history),
                   method='cobyla', options={'tol': 1.e-4, 'maxiter': 100})

In [ ]:
plt.plot(history['cost_history'])
plt.axhline(exact_ground_state_energy, color='k', linestyle='--')

結果はあまり厳密解に近くないかもしれません。

ただし、今回はいつものように「これだからNISQは」で片付けて諦めなくてもいいかもしれません。すでに2017年のハードウェアでまともな結果が得られているわけですから。

2017年のペーパーでは量子計算部分を極力効率化しています。おそらく最も効いているのは、分子軌道の基底を変換して実行的に二軌道の問題に直し、2量子ビット回路で問題を解いていることですが、今の4量子ビットのままでもアンザッツの工夫などでもっと良い結果が得られるはずです。是非自分で色々なアイディアを試してみてください。

ヒントとして、比較的に取り組みやすい改善法を挙げておきます。

- 今のアンザッツは電子数を保存しない（アンザッツが作る状態に"1"が2個でない計算基底も重ね合わさっている）が、水素分子のエネルギー固有ベクトルは電子2つの計算基底で張られる部分空間にあるはずである。パラメータ付き回路の構造を変えて、状態が任意のパラメータの値について$c_3 \ket{0011} + c_5 \ket{0101} + c_6 \ket{0110} + c_9 \ket{1001} + c_{10} \ket{1010} + c_{12} \ket{1100}$となるようにしてみる。
- アンザッツのCNOTの数を減らす。つまりCNOTが隣接量子ビット同士にのみかかるようにし、SWAPができるだけ起こらないようにする。
- パラメータの初期値を完全にランダムにせず、基底状態の近似を与えるものにする。
